In [1]:
import os

import numpy as np
import pandas as pd
import netCDF4 as nc

In [2]:
DATA_FILE_DIR = "./nasa/"

START_YEAR, END_YEAR = 2010, 2020

NUM_OF_YEARS = END_YEAR - START_YEAR

NUM_OF_MONTHS = 12

NUM_OF_DAYS = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}

In [3]:
file = nc.Dataset(DATA_FILE_DIR+'20110101.nc4')

lat = file.variables['lat'][:].filled()
lon = file.variables['lon'][:].filled()
mask = file.variables['AvgSurfT_tavg'][0].mask

LON = len(lon)
LAT = len(lat)

file.close()

In [4]:
def get_tmp(filepath):
    assert os.path.isfile(filepath), '{} does not exist!'.format(filepath)
    
    file = nc.Dataset(filepath)
    temperature = file.variables['AvgSurfT_tavg'][0]
    file.close()
    return temperature.filled(np.nan)

In [5]:
%%time
each_year_HDD = np.ndarray(shape=(NUM_OF_YEARS, LAT, LON))
each_year_MPID = np.ndarray(shape=(NUM_OF_YEARS, LAT, LON))
for year in range(START_YEAR, END_YEAR):
    print(year)
    yearly_temp = np.ndarray(shape=(365, LAT, LON))
    i = 0
    for month in range(1, NUM_OF_MONTHS+1):
        for day in range(1, NUM_OF_DAYS[month]+1):
            date = "{}{:02d}{:02d}".format(year, month, day)
            filepath = DATA_FILE_DIR + date + '.nc4'
            yearly_temp[i] = get_tmp(filepath)
            i += 1
    
    date_HDD = np.where(yearly_temp<291.15, 291.15-yearly_temp, 0) #291.15 K = 18 oC
    date_MPID = np.where(yearly_temp<253.15, 1, 0) # 253.15 K = -20 oC
    
    each_year_HDD[year-START_YEAR] = date_HDD.sum(axis=0)
    each_year_MPID[year-START_YEAR] = date_MPID.sum(axis=0)

2010


<timed exec>:14: RuntimeWarning: invalid value encountered in less
<timed exec>:15: RuntimeWarning: invalid value encountered in less


2011
2012
2013
2014
2015
2016
2017
2018
2019
Wall time: 4min 5s


In [6]:
avg_HDD = each_year_HDD.mean(axis=0)
avg_MPID = each_year_MPID.mean(axis=0)

In [39]:
pos_HDD = np.argmax(avg_HDD)
print(pos_HDD // LON, pos_HDD % LON)

556 556


In [40]:
pos_MPID = np.argmax(avg_MPID)
print(pos_MPID // LON, pos_MPID % LON)

564 563


In [41]:
avg_MPID[564, 563]

248.6

In [42]:
avg_MPID[556, 556]

242.7

In [43]:
avg_HDD[564, 563]

16595.656277465725

In [44]:
avg_HDD[556, 556]

16866.57205047599